Trying lo load a model

In [23]:
import numpy as np
import torch
from sdo.models.autocalibration1 import Autocalibration1
from torch.utils.data import DataLoader
from sdo.datasets.dimmed_sdo_dataset import DimmedSDO_Dataset
from sdo.pytorch_utilities import init_gpu, set_seed

In [13]:
path_model = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap//pipeline_results/experiments_results/brad_experiment_20/0500_brad_experiment_20_model.pth'

## Loading the model 

In [14]:
num_channels = 7
scaled_height = 128
scaled_width = 128

model = Autocalibration1(input_shape=[num_channels, scaled_height, scaled_width],
                         output_dim=num_channels)
model.eval()

Autocalibration1(
  (_conv2d1): Conv2d(7, 64, kernel_size=(3, 3), stride=(1, 1))
  (_conv2d2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (_fc1): Linear(in_features=21632, out_features=256, bias=True)
  (_fc2): Linear(in_features=256, out_features=7, bias=True)
)

In [15]:
state_dict = torch.load(path_model)
model.load_state_dict(state_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Checking the loaded model works

In [ ]:
device = init_gpu()
set_seed()

In [18]:
batch_size_test = 1
tolerance = 0.05

test_dataset = DimmedSDO_Dataset(num_channels,
                                 instr=["AIA"] * num_channels,
                                 channels=[
                                    "0094",
                                    "0131",
                                    "0171",
                                    "0193",
                                    "0211",
                                    "0304",
                                    "0335"
                                 ],
                                 yr_range=[2012, 2013],
                                 mnt_step=1,
                                 day_step=1,
                                 h_step=6,
                                 min_step=60,
                                 resolution=512,
                                 subsample=4,
                                 normalization=0,
                                 scaling=True,
                                 norm_by_orig_img_max=False,
                                 norm_by_dimmed_img_max=False,
                                 return_random_dim=False,
                                 min_alpha=0.01)
test_loader = DataLoader(test_dataset, batch_size=batch_size_test)

In [22]:
all_input = []
all_gt_output = []
all_output = []

for batch_idx, (input_data, gt_output, _) in enumerate(test_loader):
  input_data = input_data
  gt_output = gt_output
  output = model(input_data)
  
  input_data = input_data.detach().numpy()
  output = output.detach().numpy()
  gt_output = gt_output.detach().numpy()
  
  all_input.append(input_data)
  all_gt_output.append(gt_output)
  all_output.append(output)

all_input = np.array(all_input)
all_gt_output = np.array(all_gt_output)
all_output = np.array(all_output)

In [25]:
abs(all_gt_output - all_output).mean()

0.009486209

## Copy blindly from repo

In [28]:
import os
import numpy as np

import torch
from torch.optim import Adam
from torchvision import models

from sdo.viz.misc_functions import preprocess_image, recreate_image, save_image


class CNNLayerVisualization():
    """
        Produces an image that minimizes the loss of a convolution
        operation for a specific layer and filter
    """
    def __init__(self, model, selected_layer, selected_filter):
        self.model = model
        self.model.eval()
        self.selected_layer = selected_layer
        self.selected_filter = selected_filter
        self.conv_output = 0
        # Create the folder to export images if not exists
        if not os.path.exists('../generated'):
            os.makedirs('../generated')

    def hook_layer(self):
        def hook_function(module, grad_in, grad_out):
            # Gets the conv output of the selected filter (from selected layer)
            self.conv_output = grad_out[0, self.selected_filter]
        # Hook the selected layer
        self.model[self.selected_layer].register_forward_hook(hook_function)

    def visualise_layer_with_hooks(self):
        # Hook the selected layer
        self.hook_layer()
        # Generate a random image
        random_image = np.uint8(np.random.uniform(150, 180, (224, 224, 3)))
        # Process image and return variable
        processed_image = preprocess_image(random_image, False)
        # Define optimizer for the image
        optimizer = Adam([processed_image], lr=0.1, weight_decay=1e-6)
        for i in range(1, 31):
            optimizer.zero_grad()
            # Assign create image to a variable to move forward in the model
            x = processed_image
            for index, layer in enumerate(self.model):
                # Forward pass layer by layer
                # x is not used after this point because it is only needed to trigger
                # the forward hook function
                x = layer(x)
                # Only need to forward until the selected layer is reached
                if index == self.selected_layer:
                    # (forward hook function triggered)
                    break
            # Loss function is the mean of the output of the selected layer/filter
            # We try to minimize the mean of the output of that specific filter
            loss = -torch.mean(self.conv_output)
            print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data.numpy()))
            # Backward
            loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(processed_image)
            # Save image
            if i % 5 == 0:
                im_path = '../generated/layer_vis_l' + str(self.selected_layer) + \
                    '_f' + str(self.selected_filter) + '_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)

    def visualise_layer_without_hooks(self):
        # Process image and return variable
        # Generate a random image
        random_image = np.uint8(np.random.uniform(150, 180, (224, 224, 3)))
        # Process image and return variable
        processed_image = preprocess_image(random_image, False)
        # Define optimizer for the image
        optimizer = Adam([processed_image], lr=0.1, weight_decay=1e-6)
        for i in range(1, 31):
            optimizer.zero_grad()
            # Assign create image to a variable to move forward in the model
            x = processed_image
            for index, layer in enumerate(self.model):
                # Forward pass layer by layer
                x = layer(x)
                if index == self.selected_layer:
                    # Only need to forward until the selected layer is reached
                    # Now, x is the output of the selected layer
                    break
            # Here, we get the specific filter from the output of the convolution operation
            # x is a tensor of shape 1x512x28x28.(For layer 17)
            # So there are 512 unique filter outputs
            # Following line selects a filter from 512 filters so self.conv_output will become
            # a tensor of shape 28x28
            self.conv_output = x[0, self.selected_filter]
            # Loss function is the mean of the output of the selected layer/filter
            # We try to minimize the mean of the output of that specific filter
            loss = -torch.mean(self.conv_output)
            print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data.numpy()))
            # Backward
            loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(processed_image)
            # Save image
            if i % 5 == 0:
                im_path = '../generated/layer_vis_l' + str(self.selected_layer) + \
                    '_f' + str(self.selected_filter) + '_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)


ImportError: cannot import name 'preprocess_image'